In [15]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="quick-starts-ws-187595")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-187595
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-187595


In [16]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "cpu-cluster1"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found the cluster:", cpu_cluster_name)
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)    
    compute_target.wait_for_completion(show_output=True)

Found the cluster: cpu-cluster1


In [3]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C":choice(0.01,0.1,1,5,20,100), "--max_iter":choice(10,50,100)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 1, slack_factor= 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./', entry_script = 'train.py', compute_target = cpu_cluster_name, vm_size="Standard_D2_V2")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est, 
    hyperparameter_sampling=ps, 
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 20,
    max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

KeyError: 'log_files'

In [5]:
hyperdrive_run.wait_for_completion(show_output = True)

RunId: HD_34cc1182-4d88-4d08-b187-2174f8d2c6e8
Web View: https://ml.azure.com/runs/HD_34cc1182-4d88-4d08-b187-2174f8d2c6e8?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-187595/workspaces/quick-starts-ws-187595&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-02-26T19:57:36.881902][API][INFO]Experiment created<END>\n""<START>[2022-02-26T19:57:37.694819][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-02-26T19:57:38.359950][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2022-02-26T19:58:07.394675][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-02-26T19:58:07.668855][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2022-02-26T19:59:07.0795811Z][SCHEDULER][INFO]Scheduling job, id='

{'runId': 'HD_34cc1182-4d88-4d08-b187-2174f8d2c6e8',
 'target': 'cpu-cluster1',
 'status': 'Completed',
 'startTimeUtc': '2022-02-26T19:57:36.647132Z',
 'endTimeUtc': '2022-02-26T20:12:43.644552Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '627dd10c-bbd9-427e-8f5c-f07912c93c82',
  'user_agent': 'python/3.8.1 (Linux-5.4.0-1068-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': '18',
  'score': '0.9176024279210926',
  'best_child_run_id': 'HD_34cc1182-4d88-4d08-b187-2174f8d2c6e8_1',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg187595.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_34cc1182-4d88-4d08-b187-2174f8d2c6e8/azureml-logs/hyp

In [6]:
import joblib

# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_file_names())
hyperdrive_model = best_run.register_model(model_name = 'BankMarketingModel', model_path = 'outputs/model.joblib')
print(best_run)

['azureml-logs/55_azureml-execution-tvmps_1130adfddba4adaedcb9bfb305d81aced90b1daa60558113c304771bfc4ef5de_d.txt', 'azureml-logs/65_job_prep-tvmps_1130adfddba4adaedcb9bfb305d81aced90b1daa60558113c304771bfc4ef5de_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_1130adfddba4adaedcb9bfb305d81aced90b1daa60558113c304771bfc4ef5de_d.txt', 'logs/azureml/104_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']
Run(Experiment: quick-starts-ws-187595,
Id: HD_34cc1182-4d88-4d08-b187-2174f8d2c6e8_1,
Type: azureml.scriptrun,
Status: Completed)


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(url)

In [26]:
from train import clean_data
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

# Processing the df
y_df = pd.DataFrame(y,columns=['y'])
all_data = pd.concat([x,y_df], axis = 1)

x_train, x_test = train_test_split(all_data, test_size=0.2, random_state=1)

x_train.to_csv("training/train_data.csv", index = False)

data_store = ws.get_default_datastore()
data_store.upload(src_dir = "./training", target_path = 'udacity-project', overwrite = True,  show_progress = True)
train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("udacity-project/train_data.csv"))

KeyboardInterrupt: 

In [24]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    enable_early_stopping = True,
    primary_metric= "AUC_weighted",
    training_data= df,
    label_column_name="y",
    enable_onnx_compatible_models=True,
    n_cross_validations= 3,
    compute_target=compute_target)

In [20]:
!pip install azureml-sdk
!pip install azureml-core

In [25]:
# Submit your automl run
### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output = True)

ConfigException: ConfigException:
	Message: Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset]Please refer to documentation for converting to Supported types: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.dataset.dataset?view=azure-ml-py
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset]Please refer to documentation for converting to Supported types: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.dataset.dataset?view=azure-ml-py",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "training_data",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid",
                "inner_er

In [ ]:
# Retrieve and save your best automl model.
from azureml.automl.runtime.onnx_convert import OnnxConverter
best_run_model, onnx_model= remote_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, file_path="./automl_model.onnx")

In [ ]:
#Delete compute_target
compute_target.delete()